In [1]:
import numpy as np
import math
import sklearn.linear_model
import sklearn.metrics

import time

from matplotlib import pyplot as plt
%matplotlib inline

import load
import cnn

Using Theano backend.
/home/michel/.virtualenvs/solar/lib/python3.4/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
train_ids, val_ids, test_ids, train_labels, val_labels, test_labels = load.get_sets()

In [3]:
cnn_train = cnn.get_cnn_features(train_ids)
cnn_val = cnn.get_cnn_features(val_ids)
cnn_test = cnn.get_cnn_features(test_ids)

In [4]:
class_weight=None # or 'balanced'

## CNN

In [ ]:
n_C = 20
C_list = np.logspace(-10, 10, n_C)
errors = np.zeros(n_C)
for i, C in enumerate(C_list):
    model = sklearn.linear_model.LogisticRegression(penalty='l2', C=C, class_weight=class_weight)
    t = time.time()
    model.fit(cnn_train, train_labels)
    print("LR took %f seconds."%(time.time()-t))
    predictions = model.predict(cnn_val)
    errors[i] = (val_labels!=predictions).sum()/len(val_labels)

In [ ]:
plt.plot(C_list, errors)
plt.xscale('log')

In [6]:
#i_opt = np.argmin(errors)
C_opt = 0.000018#C_list[i_opt]
print('optimal C : %f'%C_opt)
model_cnn = sklearn.linear_model.LogisticRegression(penalty='l2', C=C_opt, class_weight=class_weight)
t = time.time()
model_cnn.fit(cnn_train, train_labels)
print("LR took %f seconds."%(time.time()-t))
predictions = model_cnn.predict(cnn_test)
err = (test_labels!=predictions).sum()/len(test_labels)

optimal C : 0.000018
LR took 33.704868 seconds.


In [7]:
err

0.21041666666666667

In [8]:
cnn_proba_val = model_cnn.predict_proba(cnn_val)[:,:-1]
cnn_proba_test = model_cnn.predict_proba(cnn_test)[:,:-1]

## LR

In [9]:
l = 6
color = False
train_reduc, val_reduc, test_reduc = load.load_all_data(train_ids, val_ids, test_ids, l, color)

In [10]:
model_lr = sklearn.linear_model.LogisticRegression(penalty='l2', C=1e10, class_weight=class_weight)
model_lr.fit(train_reduc, train_labels)
predictions = model_lr.predict(val_reduc)
error = (val_labels!=predictions).sum()/len(val_labels)
error

0.39966624947851481

In [11]:
reduc_proba_val = model_lr.predict_proba(val_reduc)[:,:-1]
reduc_proba_test = model_lr.predict_proba(test_reduc)[:,:-1]

In [12]:
val_mix = np.concatenate([cnn_proba_val, reduc_proba_val], axis=1)
test_mix = np.concatenate([cnn_proba_test, reduc_proba_test], axis=1)

In [13]:
model_ensemble = sklearn.linear_model.LogisticRegression(penalty='l2', C=1e10, class_weight=class_weight)
model_ensemble.fit(val_mix, val_labels)
predictions = model_ensemble.predict(test_mix)
error = (test_labels!=predictions).sum()/len(test_labels)
error

0.19875000000000001

In [14]:
probas = model_ensemble.predict_proba(test_mix)

## Confusion matrix

In [15]:
sklearn.metrics.confusion_matrix(test_labels, predictions)

array([[957,  22,  10,  51],
       [ 26, 426,  10,  69],
       [ 11,  20, 208,  23],
       [126,  86,  23, 332]])

## Confidence

In [ ]:
confidence = np.max(probas, axis=1)

In [ ]:
plt.hist(confidence, bins=100)

In [ ]:
N = len(confidence)
confidence_order = np.argsort(-confidence)
sorted_error = (predictions[confidence_order]!=test_labels[confidence_order])
nb_errors = np.cumsum(sorted_error)
error_rate = nb_errors/np.arange(N)

In [ ]:
plt.plot(np.arange(N)/N, error_rate)
plt.xlabel('% classified')
plt.ylabel('% error')

In [ ]:
def custom_convolution(a, kernel):
    n = len(a)
    k = int((len(kernel)-1)/2)
    
    conv = np.zeros(n)
    for i in range(n):
        imin_kernel = max(0, k-i)
        imax_kernel = min(2*k+1, k+n-i)
        imin_a = max(0, i-k)
        imax_a = min(n, k+1+i)
        conv[i] = (a[imin_a:imax_a]*kernel[imin_kernel:imax_kernel]).sum() / (kernel[imin_kernel:imax_kernel].sum())
    
    return conv

In [ ]:
sigma = 40.
kernel_size = int(math.floor(3.5*sigma))
kernel = np.exp(-np.power(np.arange(2*kernel_size+1)-kernel_size, 2.)/(2*np.power(sigma, 2.)))
kernel /= kernel.sum()

smooth_error = custom_convolution(sorted_error, kernel)
plt.figure()
plt.plot(smooth_error)
plt.xlabel('confidence rank')
plt.ylabel('% error')

## Save model

In [18]:
import pickle
pickle.dump((model_cnn, model_lr, model_ensemble), open('models.pkl', 'wb'))